<a href="https://colab.research.google.com/github/ahzaidy/Programs/blob/main/CPSC_5440_HW31.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from skimage.metrics import structural_similarity as ssim
from sklearn.metrics import mean_squared_error

# Set dataset paths
train_dir = "/content/drive/My Drive/train1/"
test_dir = "/content/drive/My Drive/test1/"

# Function to load and preprocess images
def load_images(directory, size=(56, 56)):
    images = []
    filenames = os.listdir(directory)
    for file in filenames:
        img = plt.imread(os.path.join(directory, file))  # Load image as np.array
        img = np.resize(img, (*size, 3)) / 255.0  # Normalize and resize
        images.append(img)
    return np.array(images)

# Load images
x_train = load_images(train_dir)
x_test = load_images(test_dir)

# Convert to PyTorch tensors
x_train = torch.tensor(x_train, dtype=torch.float32).permute(0, 3, 1, 2)
x_test = torch.tensor(x_test, dtype=torch.float32).permute(0, 3, 1, 2)

# Add noise to training data
noise_factor = 0.5
x_train_noisy = x_train + noise_factor * torch.randn_like(x_train)
x_test_noisy = x_test + noise_factor * torch.randn_like(x_test)
x_train_noisy = torch.clamp(x_train_noisy, 0., 1.)
x_test_noisy = torch.clamp(x_test_noisy, 0., 1.)

# Define Autoencoder model
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()

        # Encoder
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        # Decoder
        self.decoder = nn.Sequential(
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Upsample(scale_factor=2, mode='nearest'),
            nn.Conv2d(128, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Upsample(scale_factor=2, mode='nearest'),
            nn.Conv2d(64, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Upsample(scale_factor=2, mode='nearest'),
            nn.Conv2d(32, 3, kernel_size=3, padding=1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

# Initialize model, loss function, and optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Autoencoder().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
epochs = 50
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(x_train_noisy.to(device))
    loss = criterion(outputs, x_train.to(device))
    loss.backward()
    optimizer.step()
    print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")

# Save model
torch.save(model.state_dict(), "/content/drive/My Drive/image_upscaler.pth")

# Evaluate model
model.eval()
reconstructed = model(x_test_noisy.to(device)).cpu().detach().numpy()

# Compute MSE and SSIM
mse_values = []
ssim_values = []
num_images = 50  # Reduced for demonstration purposes

fig, axes = plt.subplots(10, 3, figsize=(10, 20))
for i in range(num_images):
    original = x_test[i].permute(1, 2, 0).numpy()
    noisy = x_test_noisy[i].permute(1, 2, 0).numpy()
    recon = reconstructed[i].transpose(1, 2, 0)

    mse = mean_squared_error(original.flatten(), recon.flatten())
    mse_values.append(mse)
    ssim_index = ssim(original, recon, data_range=1.0, channel_axis=-1)
    ssim_values.append(ssim_index)

    axes[i, 0].imshow(original)
    axes[i, 0].set_title("Original")
    axes[i, 0].axis("off")

    axes[i, 1].imshow(noisy)
    axes[i, 1].set_title("Noisy")
    axes[i, 1].axis("off")

    axes[i, 2].imshow(recon)
    axes[i, 2].set_title(f"Reconstructed\nMSE: {mse:.4f}\nSSIM: {ssim_index:.4f}")
    axes[i, 2].axis("off")

plt.tight_layout()
plt.show()

# Print average MSE and SSIM
print(f"Average MSE: {np.mean(mse_values):.4f}")
print(f"Average SSIM: {np.mean(ssim_values):.4f}")


In [ ]:
!pip install skimage